In [17]:
# Librerías
import numpy as np
import pandas as pd
import openpyxl as wb
from datetime import datetime as dt
import os

Extracción de data y definición de hojas como dF

In [18]:
# La ruta se modifica según la ubicación del archivo en el equipo
ruta = r'C:\Users\Madiaz\OneDrive - ZENTRIA\Documentos\Información Michael Dìaz\---PERSONAL\1. ESPECIALIZACIÓN\--- MODULO 4 - ANTEPROYECTO Y TRABAJO DE GRADO\Reporte de prueba.xlsx'

In [19]:
# Carga de datos de las hojas del archivo Excel
df_rotacion = pd.read_excel(ruta,sheet_name="rotacion") # Datos de rotación de inventario entre los meses exportados
df_ajustes = pd.read_excel(ruta,sheet_name="ajustes") # Datos de ajustes de inventario entre los meses exportados
df_inventario = pd.read_excel(ruta,sheet_name="inventario") # Datos de inventario al corte del exporte
df_almacenes = pd.read_excel(ruta,sheet_name="almacenes") # Datos de almacenes creados en el sistema
df_oc_ce = pd.read_excel(ruta,sheet_name="oc_ce") # Datos de órdenes de compra (solicitudes a proveedores) y comprobantes de entrada (recepciones de pedidos) entre los meses exportados
df_maestro = pd.read_excel(ruta,sheet_name="maestro") # Datos del maestro de productos (insumos y medicamentos), para la identificación de códigos de los productos
df_pronostico = pd.DataFrame() # DF vacío para el pronóstico de demanda futura, se llenará en el proceso ETL para luego usar con el modelo

Llaves relacionales entre tablas

In [20]:
# Columna nueva para cada DF que hace una llave única por almacén y producto. Estas son las que nos van a ayudar a hacer el merge entre tablas
df_rotacion['llave'] = df_rotacion['CODIGO_ALMACEN'].astype(str) + '-' + df_rotacion['CODIGO_PADRE'].astype(str)
df_inventario['llave'] = df_inventario['CODIGO_ALMACEN'].astype(str) + '-' + df_inventario['CODIGO_PADRE'].astype(str)
df_ajustes['llave'] = df_ajustes['CODIGO_ALMACEN'].astype(str) + '-' + df_ajustes['CODIGO_PADRE'].astype(str)

# Traer código padre de productos a tabla de OC_CE ya que en oc_ce no existe la columna de código padre
bus_padre_maestro = pd.merge(df_oc_ce,df_maestro[['CODIGO_PRODUCTO', 'CODIGO_PADRE']],on='CODIGO_PRODUCTO',how='left')
df_oc_ce['CODIGO_PADRE'] = bus_padre_maestro['CODIGO_PADRE']
df_oc_ce['llave'] = df_oc_ce['CODIGO_ALMACEN'].astype(str) + '-' + df_oc_ce['CODIGO_PADRE'].astype(str)

# Traer almacén de entrega a la tabla de rotación, para luego llevarlo al DF final del pronóstico
bus_almacen_entrega = pd.merge(df_rotacion.astype(str),df_almacenes[['CODIGO_ALMACEN', 'SEDE_DE_ENTREGA']].astype(str),on='CODIGO_ALMACEN',how='left')
df_rotacion['ALMACEN_ENTREGA'] = bus_almacen_entrega['SEDE_DE_ENTREGA']

Cálculo de variables

In [21]:
######################################################################
# Para OC_CE

# Calcular días entre emisión de factura y generación de Comprobante de Entrada
dias_factura_compentr = df_oc_ce['FECHA'] - df_oc_ce['FECHA_FACTURA']
df_oc_ce['DIAS_ENTRE_FACT_Y_CE'] = dias_factura_compentr

# Calcular dias promedio y máximo de entrega de factura para calculo posterior de Stock de seguridad
df_oc_ce['dias_prom_entrega'] = df_oc_ce.groupby('llave')['DIAS_ENTRE_FACT_Y_CE'].mean()
df_oc_ce['dias_max_entrega'] = df_oc_ce.groupby('llave')['DIAS_ENTRE_FACT_Y_CE'].max()

######################################################################
# PARA ROTACION

# Calcular rotación promedio diaria
col_promedio = ['Ene_23','Feb_23','Mar_23','Abr_23','May_23','Jun_23','Jul_23',
               'Ago_23','Sep_23','Oct_23','Nov_23','Dic_23','Ene_24','Feb_24',
               'Mar_24','Abr_24','May_24','Jun_24','Jul_24','Ago_24','Sep_24',
               'Oct_24','Nov_24','Dic_24']
# Verificar qué columnas existen realmente en el DataFrame
columnas_validas = [col for col in col_promedio if col in df_rotacion.columns]
# Calcular RPD (rotación promedio diaria) considerando solo las columnas válidas
df_rotacion['RPD'] = (df_rotacion[columnas_validas].mean(axis=1, skipna=True)/30)
#df_rotacion.head()


# Variación de consumos. Es un amortiguador para evitar que el modelo reaccione a cambios bruscos en el consumo. Es usado en el holt-winters
# La variación se calcula como el valor absoluto de 1 menos el cociente entre el consumo proyectado por RPD y el último consumo registrado
# El consumo proyectado se calcula como RPD * 30 (promedio mensual). No lo vamos a usar porque se usa internamente en el modelo, pero si
# posiblemente para validación de variaciones entre el holt-winters y el modelo que usemos para el pronóstico
columnas_rango = ['Ene_23','Feb_23','Mar_23','Abr_23','May_23','Jun_23','Jul_23',
               'Ago_23','Sep_23','Oct_23','Nov_23','Dic_23','Ene_24','Feb_24',
               'Mar_24','Abr_24','May_24','Jun_24','Jul_24','Ago_24','Sep_24',
               'Oct_24','Nov_24','Dic_24']
ultimo_consumo = df_rotacion[columnas_rango].ffill(axis=1).iloc[:, -1]
df_rotacion['variacion_consumo'] = np.abs(1 - ((df_rotacion['RPD']*30)/ultimo_consumo))

Construcción de tabla final para el pronóstico

In [22]:
# DF final del pronóstico con sus respectivos campos
df_pronostico = df_rotacion[['llave', 'ALMACEN_ENTREGA', 'CODIGO_PADRE', 'DESCRIPCION_PADRE', 'TIPO_PRODUCTO', 'RPD']].copy()
df_pronostico['llave'] = df_pronostico['llave'].astype(str)
df_pronostico['RPD'] = df_pronostico['RPD'].astype(float).round(2)

# Merge con inventario para traer la cantidad disponible al corte del reporte
df_pronostico = pd.merge(df_pronostico,df_inventario[['llave', 'CANTIDAD']].rename(columns={'CANTIDAD': 'Disponible'}),on='llave',how='left')
df_pronostico['Disponible'] = df_pronostico['Disponible'].fillna(0).astype(float).round(2)

# Merge con OC_CE para traer los días promedio y máximo de entrega, que nos ayudarán a calcular el stock de seguridad
df_pronostico = pd.merge(df_pronostico,df_oc_ce[['llave', 'dias_max_entrega', 'dias_prom_entrega']],on='llave',how='left')

# Calcular: días de inventario, stock de seguridad y punto de reorden 
df_pronostico['dias_inventario'] = np.where(df_pronostico['Disponible'] > 0,(df_pronostico['RPD'] / df_pronostico['Disponible']).round(2),0)
df_pronostico['stock_seguridad'] = ((df_pronostico['dias_max_entrega'].fillna(0) - df_pronostico['dias_prom_entrega'].fillna(0)) * df_pronostico['RPD']).round(2)
df_pronostico['punto_reorden'] = ((df_pronostico['RPD'] * df_pronostico['dias_prom_entrega'].fillna(0)) + df_pronostico['stock_seguridad']).round(2)

print(df_pronostico.head())

# PARA EL HOLT WINTERS
# Columna #: Pronóstico de consumo próximo
# df_pronostico['Pronóstico'] = (ultimo_consumo + (df_rotacion['variacion_consumo']*(ultimo_consumo)))

# Columna 12: Pedido a realizar
#df_pronostico['pedido_realizar'] = np.where(df_pronostico['Disponible'] < 
#                                            (df_pronostico['RPD'] * (df_pronostico['dias_inventario'] + df_pronostico['stock_seguridad'] + df_pronostico['punto_reorden'])),
#                                            (df_pronostico['RPD'] * 30) + df_pronostico['stock_seguridad'],'No comprar')

           llave             ALMACEN_ENTREGA CODIGO_PADRE  \
0  174-L01XX3201            PEI - MARAYA CRM    L01XX3201   
1  002-L01XE3801                         NaN    L01XE3801   
2  191-N01AX1001  MAN - AMAN HOSPITALIZACIÓN    N01AX1001   
3     035-I03099                         NaN       I03099   
4  073-N02AE0102                         NaN    N02AE0102   

                                   DESCRIPCION_PADRE TIPO_PRODUCTO   RPD  \
0  BORTEZOMIB 3,5 MG POLVO ESTERIL PARA RECONSTIT...   MEDICAMENTO  0.00   
1                        COBIMETINIB X 20 MG TABLETA   MEDICAMENTO  0.23   
2  PROPOFOL 200 MG X 20 ML (10MG/ML) SOLUCION INY...   MEDICAMENTO  0.00   
3                  SONDA DE SUCCION ABIERTA N° 14 FR       INSUMOS  0.30   
4  BUPRENORFINA (52,5 MCG/H) X 30 MG PARCHE TRANS...   MEDICAMENTO  0.41   

   Disponible  dias_max_entrega  dias_prom_entrega  dias_inventario  \
0         0.0               NaN                NaN              0.0   
1         0.0               NaN   

In [24]:
# Esto se usará cuándo se implemente el modelo de pronóstico y sus comparativos. Por ahora, solo es informativo

ruta_impresion = r'C:\Users\Madiaz\Desktop'
fecha_hora = dt.now().strftime('%Y%m%d_%H%M%S')
nombre_completo = os.path.join(ruta_impresion, f'reporte_compras_{fecha_hora}.xlsx')
df_pronostico.to_excel(nombre_completo,sheet_name='Compras',index=False)